In [ ]:
import tensorflow as tf

tf.test.is_gpu_available()  # should return True
tf.config.list_physical_devices('GPU')

In [ ]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten, Conv2D, MaxPooling2D, AvgPool2D # new!
import tensorflow as tf
import random
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [ ]:
# Define the text labels
fashion_mnist_labels = ["T-shirt/top",  # index 0
                        "Trouser",      # index 1
                        "Pullover",     # index 2 
                        "Dress",        # index 3 
                        "Coat",         # index 4
                        "Sandal",       # index 5
                        "Shirt",        # index 6 
                        "Sneaker",      # index 7 
                        "Bag",          # index 8 
                        "Ankle boot"]   # index 9

# Image index, you can pick any number between 0 and 59,999
img_index = random.randint(0,60000)

# y_train contains the lables, ranging from 0 to 9
label_index = y_train[img_index]

# Print the label, for random example
print ("y [" + str(img_index) + "]= " + str(label_index) + " " +(fashion_mnist_labels[label_index]))

# # Show one of the images from the training dataset
plt.imshow(x_train[img_index], cmap='gray_r')

In [ ]:
# A single pixel is of type Integer
type (x_train[0][0][0])

In [ ]:
# A complete image is stored as numpy Array
type (x_train[img_index])

In [ ]:
# We received 28x28 pixel images
x_train[img_index].shape

In [ ]:
# In PyCharm we can display this 28x28 Image showing its Integer Numbers
x_train[img_index]

In [ ]:
# We want to reshape all 60.000 images (and 10.000 test images) in order to give them one color-layer (not 3 color layers)
x_train = x_train.reshape(60000, 28, 28, 1).astype('float32')
x_test = x_test.reshape(10000, 28, 28, 1).astype('float32')

In [ ]:
# We normalize Gray Scale to float 0..1
x_train /= 255
x_test /= 255

In [ ]:
# We now have an extra dimension for the gray layer.
# The numbers stored ar floats
type(x_train[0][0][0][0])

In [ ]:
# An image is still an numpy array
type(x_train[img_index])

In [ ]:
# Shape of a single image no longer is 28x28 but 28x28x1 (the gray layer)
x_train[img_index].shape

In [ ]:
# we still can display this gray layer in PyCharm
x_train[img_index].reshape(28,28)

In [ ]:
# We need to one-hot-encode our Y values
n_classes = 10
y_train = keras.utils.to_categorical(y_train, n_classes)
y_test = keras.utils.to_categorical(y_test, n_classes)

In [ ]:
# a simple LeNet implementation as found on d2l.ai
model = Sequential([
            Conv2D(filters=6, kernel_size=5, activation='sigmoid', padding='same', input_shape=(28,28,1)),
            AvgPool2D(pool_size=2, strides=2),
            Conv2D(filters=16, kernel_size=5, activation='sigmoid'),
            AvgPool2D(pool_size=2, strides=2),
            Flatten(),
            Dense(120, activation='sigmoid'),
            Dense(84, activation='sigmoid'),
            Dense(n_classes, activation='softmax')])

In [ ]:
# we can display the model summary
model.summary()

In [ ]:
# Tensorflow allows to creat a visual representation of our model.
from tensorflow.keras.utils import plot_model

plot_model(model, to_file='../img/model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
# we define two so called "callback" object.
# these objects can be injected to the training process.
# here: 
#   - the checkpointer callback collects the bestperforming model and stores the respective model parameters
#   - the history callback collects all metrics from all epochs
from keras.callbacks import ModelCheckpoint
from keras.callbacks import History

checkpointer = ModelCheckpoint(filepath='model.weights.best.hdf5', verbose = 1, save_best_only=True)                     
history = History()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
result = model.fit(x_train, y_train, batch_size=128, epochs=10, verbose=1
                   #, validation_data=(x_valid, y_valid)
                   , validation_split=0.2
                   , callbacks = [history, checkpointer])

In [ ]:
history.history['loss']

In [ ]:
fig, ax1 = plt.subplots()
ax1.plot(history.history['loss'], color='red', linestyle='--')
ax1.plot(history.history['val_loss'], color='green', linestyle='--')
plt.title('model performance')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train loss', 'val loss', 'train acc', 'varl acc'], loc='upper left')

ax2 = ax1.twinx()

ax2.plot(history.history['accuracy'])
ax2.plot(history.history['val_accuracy'])
ax2.set_ylabel('accuracy')
ax2.legend(['train acc', 'val acc'], loc='lower left')

In [ ]:
# Load the weights with the best validation accuracy
model.load_weights('model.weights.best.hdf5')

In [ ]:
# Evaluate the model on test set
score = model.evaluate(x_test, y_test, verbose=0)

# Print test accuracy
print('\n', 'Test accuracy:', score[1])

In [ ]:
y_hat = model.predict(x_test)


In [ ]:
# Plot a random sample of 15 test images, their predicted labels and ground truth
figure = plt.figure(figsize=(20, 8))
for i, index in enumerate(np.random.choice(x_test.shape[0], size=15, replace=False)):
    ax = figure.add_subplot(3, 5, i + 1, xticks=[], yticks=[])
    # Display each image
    ax.imshow(np.squeeze(x_test[index]), cmap='gray_r')
    predict_index = np.argmax(y_hat[index])
    true_index = np.argmax(y_test[index])
    # Set the title for each image
    ax.set_title("{} ({})".format(fashion_mnist_labels[predict_index], 
                                  fashion_mnist_labels[true_index]),
                                  color=("green" if predict_index == true_index else "red"))

In [ ]:
tf.test.is_gpu_available()  # should return True

# other models

In [ ]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(n_classes, activation='softmax'))

# todos

- zusätzliche Layer, wie dropout, normalization, regularisierung,...
- augmentierung
- transfer learning
- überblick über architekturen
- metriken
- balancierte / unbalancierte sets
- Auto tuner https://github.com/keras-team/keras-tuner
- 

# denkbare aufgaben
- nachvollziehen: https://keras.io/guides/transfer_learning/
- 

# NLP
- https://www.tensorflow.org/hub
- 